In [1]:
import pandas as pd
import numpy as np
import datetime
import os

os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Planner_Requests/Past_5_quarters_new_sign_up_no_purchase'

In [2]:
def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)

In [3]:
new_sign_ups=pd.read_csv("/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Year/All Rewards Members 2018-02-04 - 2019-05-04.zip",
                         dtype=str,compression="zip",sep="|",usecols=['customer_id_hashed','email_address_hash','sign_up_date','customer_zip_code'])

new_sign_ups=new_sign_ups.sort_values(['customer_id_hashed','email_address_hash','sign_up_date'],ascending=[True,True,False])
new_sign_ups=new_sign_ups.drop_duplicates("customer_id_hashed").drop_duplicates("email_address_hash")

In [17]:
Quarter_start_date_2018Q1=datetime.date(2018,2,4)
all_5_quarters_start=[Quarter_start_date_2018Q1 +datetime.timedelta(days=13*7*x) for x in range(5)]

quarter_i=1

df_all_dates=pd.DataFrame()
for date_quarter_start in all_5_quarters_start:
    if quarter_i>=5:
        year=str(2019)
    else:
        year=str(2018)
    quarter=year+"_Q"+str(quarter_i%4)
    quarter=quarter.replace("_Q0","_Q4")
    for i in range(7*13):
        sign_up_date=date_quarter_start+datetime.timedelta(days=i)
        df=pd.DataFrame({"sign_up_date":str(sign_up_date)},index=[quarter])
        df_all_dates=df_all_dates.append(df)
    quarter_i+=1
df_all_dates=df_all_dates.reset_index().rename(columns={"index":"quarter"})

In [5]:
hist_item_sales_2018=list(recursive_file_gen("/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/"))
hist_item_sales_2018=[x for x in hist_item_sales_2018 if ".txt" in x]
df_item_sales=pd.DataFrame({"file_path":hist_item_sales_2018})
df_item_sales['week_end_dt']=df_item_sales['file_path'].apply(lambda x: x.split("MediaStormDailySalesHistory")[1][:8])
####
all_item_sales_2019=list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
all_item_sales_2019=[x for x in all_item_sales_2019 if "aily" in x]
all_item_sales_2019=[x for x in all_item_sales_2019 if ".txt" in x]

df_item_sales_2019=pd.DataFrame({"file_path":all_item_sales_2019})
df_item_sales_2019['week_end_dt']=df_item_sales_2019['file_path'].apply(lambda x: x.split("/MediaStorm_")[1][:10].replace("-",""))
df_item_sales_2019=df_item_sales_2019[df_item_sales_2019['week_end_dt']>df_item_sales['week_end_dt'].max()]
####
df_item_sales=df_item_sales.append(df_item_sales_2019).sort_values("week_end_dt")
df_item_sales.shape

(43, 2)

In [6]:
hist_subclass_sales=list(recursive_file_gen("/home/jian/BigLots/hist_daily_data_subclasslevel/"))
hist_subclass_sales=[x for x in hist_subclass_sales if ".txt" in x]
df_subclass_sales=pd.DataFrame({"file_path":hist_subclass_sales})
df_subclass_sales['week_end_dt']=df_subclass_sales['file_path'].apply(lambda x: x.split("/MediaStormDailySales_week_ending_")[1][:10].replace("-",""))

df_subclass_sales=df_subclass_sales[df_subclass_sales['week_end_dt']<df_item_sales['week_end_dt'].min()]
df_subclass_sales=df_subclass_sales[df_subclass_sales['week_end_dt']>=df_all_dates['sign_up_date'].apply(lambda x: x.replace("-","")).min()]
df_subclass_sales.shape
###

all_daily_sales_2018=list(recursive_file_gen("/home/jian/BigLots/2018_by_weeks/"))
all_daily_sales_2018=[x for x in all_daily_sales_2018 if "aily" in x]
all_daily_sales_2018=[x for x in all_daily_sales_2018 if ".txt" in x]

df_subclass_sales_2018=pd.DataFrame({"file_path":all_daily_sales_2018})
df_subclass_sales_2018['week_end_dt']=df_subclass_sales_2018['file_path'].apply(lambda x: x.split("/MediaStorm_")[1][:10].replace("-",""))
df_subclass_sales_2018=df_subclass_sales_2018[df_subclass_sales_2018['week_end_dt']<df_item_sales['week_end_dt'].min()]
df_subclass_sales_2018=df_subclass_sales_2018[df_subclass_sales_2018['week_end_dt']>=df_all_dates['sign_up_date'].apply(lambda x: x.replace("-","")).min()]
df_subclass_sales_2018=df_subclass_sales_2018[df_subclass_sales_2018['week_end_dt']>df_subclass_sales['week_end_dt'].apply(lambda x: x.replace("-","")).min()]
###
df_subclass_sales=df_subclass_sales.append(df_subclass_sales_2018).sort_values("week_end_dt")
df_subclass_sales.shape


(26, 2)

In [7]:
list_all_daily_sales_up_to_20190601=df_subclass_sales['file_path'].unique().tolist()+df_item_sales['file_path'].unique().tolist()
print(len(list_all_daily_sales_up_to_20190601))

i_counter=0

df_all_shoppers=pd.DataFrame()
for file_path in list_all_daily_sales_up_to_20190601:
    df=pd.read_table(file_path,dtype=str,sep="|",usecols=['customer_id_hashed','transaction_dt'])
    df=df[~pd.isnull(df['customer_id_hashed'])]
    df=df.drop_duplicates()
    df_all_shoppers=df_all_shoppers.append(df)
    i_counter+=1
    if i_counter%10==2:
        print(i_counter,datetime.datetime.now())

df_all_shoppers_min=df_all_shoppers.sort_values(['customer_id_hashed','transaction_dt'],ascending=[True,True])
df_all_shoppers_min=df_all_shoppers_min.drop_duplicates("customer_id_hashed").rename(columns={"transaction_dt":"transaction_dt_min"})

df_all_shoppers_max=df_all_shoppers.sort_values(['customer_id_hashed','transaction_dt'],ascending=[True,False])
df_all_shoppers_max=df_all_shoppers_max.drop_duplicates("customer_id_hashed").rename(columns={"transaction_dt":"transaction_dt_max"})

df_all_shoppers=pd.merge(df_all_shoppers_min,df_all_shoppers_max,on=['customer_id_hashed'])

69
2 2019-06-05 15:20:19.642519
12 2019-06-05 15:23:25.829003
22 2019-06-05 15:26:40.943537
32 2019-06-05 15:30:58.412810
42 2019-06-05 15:36:14.266157
52 2019-06-05 15:42:50.222262
62 2019-06-05 15:48:31.490033


In [8]:
new_sign_ups=pd.merge(new_sign_ups,df_all_shoppers,on="customer_id_hashed",how="left")
new_sign_ups=pd.merge(new_sign_ups,df_all_dates,on="sign_up_date",how="left")


In [9]:
new_sign_ups['Shop_Label']=np.where(pd.isnull(new_sign_ups['transaction_dt_min']),"No_Trans","Shoppers")

In [23]:
for quarter,group in new_sign_ups.groupby(['quarter']):
    df=group[group['Shop_Label']=="No_Trans"]
    df=df[['customer_id_hashed','email_address_hash','customer_zip_code','quarter']]
    df=df.rename(columns={"quarter":"segment"})
    segment_value="No_Trans_signed_up_in_"+quarter
    df['segment']=segment_value
    df.to_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Planner_Requests/Past_5_quarters_new_sign_up_no_purchase/BL_new_sign_ups_in_"+quarter+"_No_Trans_JL_"+str(datetime.datetime.now().date())+".csv",index=False)
    print(quarter,len(df))
    

2018_Q1 253490
2018_Q2 249591
2018_Q3 416867
2018_Q4 839981
2019_Q1 515460


In [32]:
summary=new_sign_ups.groupby(['Shop_Label','quarter'])['customer_id_hashed'].nunique().to_frame().reset_index()
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Planner_Requests/Past_5_quarters_new_sign_up_no_purchase/BL_summary_5_quarters_new_rewards_no_trans_yet_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
summary.to_excel(writer,"summary_5_quarters_count",index=False)

for quarter,group in new_sign_ups.groupby(['quarter']):
    df=group[group['Shop_Label']=="No_Trans"].head(1000)
    df=df[['customer_id_hashed','email_address_hash','customer_zip_code','quarter']]
    df=df.rename(columns={"quarter":"segment"})
    segment_value="No_Trans_signed_up_in_"+quarter
    df['segment']=segment_value
    df.to_excel(writer,"sample_signed_"+quarter,index=False)
    print(quarter,len(df))

writer.save()

2018_Q1 1000
2018_Q2 1000
2018_Q3 1000
2018_Q4 1000
2019_Q1 1000


In [30]:
new_sign_ups[new_sign_ups['customer_id_hashed']=="001081be4ed7e8a9f368ab57e65ad81a94f90a9447485eadee6eb128386a6b6b"]

,customer_id_hashed,email_address_hash,sign_up_date,customer_zip_code,transaction_dt_min,transaction_dt_max,Shop_Label,quarter
2035,001081be4ed7e8a9f368ab57e65ad81a94f90a9447485e...,9b7541bfe0622fb9c7d6c9af5da51fa586ec3d2fa1aa3f...,2019-02-26,NaN,NaN,NaN,No_Trans,2019_Q1


In [31]:
df.head(2)

,customer_id_hashed,email_address_hash,customer_zip_code,segment
9,00001766759bd0e8524c71458b12ae242fd19358c6e3c1...,dba230ccf004d66bdb7064f8665967c217958dd13f6350...,NaN,No_Trans_signed_up_in_2019_Q1
22,00002626764ee23b4f28532d66ea283681502198037b53...,0b8036e2de96691601ae00514c8d04896170224f47a47a...,NaN,No_Trans_signed_up_in_2019_Q1
